In [18]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [19]:
df = pd.read_csv(os.path.join('csvFiles','train.csv','train.csv'))

In [3]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,racism
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


PREPROCESSING DATA

In [20]:
from tensorflow import keras
from keras.layers import TextVectorization

In [29]:
X = df['comment_text'] #all the act text
y = df[df.columns[2:]].values #the lavels of said text (toxic, super toxic, etc)

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       ...,
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [30]:
MAX_FEATURES = 200000 # number of words in the vocab

In [31]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES, output_sequence_length=1800, output_mode='int')

In [32]:
vectorizer.adapt(X.values)


In [33]:
vectorized_text = vectorizer(X.values)

In [34]:
#MCSHBAP - map, cache, shuffle, batch, prefetch, from_tensor_slice, list_file
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) #helps bottlenecks

In [35]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

CREATE SEQUENTIAL MODEL

In [36]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding
from keras.callbacks import EarlyStopping

In [37]:
model = Sequential()
# Create embedding layer
model.add(Embedding(MAX_FEATURES+1, 32))
# Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(32, activation='tanh')))
# Feature extractor Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5)),
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5)),
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5)),
# Final layer
model.add(Dense(6, activation='sigmoid'))

In [38]:
model.compile(loss='BinaryCrossentropy', optimizer='adam')

In [39]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirection  (None, 64)                16640     
 al)                                                             
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                        

In [40]:
early_stopping = EarlyStopping(patience=10, min_delta=0.001, restore_best_weights=True)

In [58]:
history = model.fit(train, epochs=10, callbacks=[early_stopping], validation_data=val, verbose=1)

Epoch 1/10


ValueError: in user code:

    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py", line 1209, in compute_loss
     

MAKING PREDICTIONS

In [43]:
input_text = vectorizer('I WILL KILL YOU')

In [44]:
batch = test.as_numpy_iterator().next()

In [45]:
batch_X, batch_y = test.as_numpy_iterator().next()

In [46]:
(model.predict(batch_X) > 0.5).astype(int)

1/1 [==============================] - 2s 2s/step


array([[0, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 0]])

In [47]:
res = model.predict(np.expand_dims(input_text,0))

1/1 [==============================] - 0s 93ms/step


EVALUATE THE MODEL

In [48]:
from keras.metrics import Precision, Recall, CategoricalAccuracy

In [49]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [25]:
for batch in test.as_numpy_iterator():
    # Unpacking each individual batch
    X_true, y_true = batch
    # yhat is our prediction
    yhat = model.predict(X_true)

    # Flatten our predictions
    y_true = y_true.flatten()
    yhat = yhat.flatten()

    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

1/1 [==============================] - 0s 93ms/step


In [50]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

Precision: 0.0, Recall:0.0, Accuracy:0.0


Test and Gradio

In [51]:
# pip install gradio, and jinja2
import gradio as gr
import tensorflow as tf


In [28]:
model.save('oops.h5')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [52]:
model = tf.keras.models.load_model('toxic_model_ten_epochs.h5')

In [53]:
input_str = vectorizer('jerk')

In [54]:
res = model.predict(np.expand_dims(input_str,0))
res

1/1 [==============================] - 1s 1s/step


array([[0.8706537 , 0.01240917, 0.2955679 , 0.02620247, 0.3649424 ,
        0.06257385]], dtype=float32)

In [55]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)

    text = ''
    for idx, col in enumerate(df.columns[[2,5,7]]): # I only really want toxic, insult, and racist
        text += '{}: {}\n'.format(col, results[0][idx] > 0.5)
    
    return text

In [56]:
interface = gr.Interface(fn=score_comment, 
                         inputs=gr.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')

In [57]:
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://8576a0d6db0295fbf3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 [==============================] - 0s 130ms/step
